In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import tifffile as tiff

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import hyper_img as hyper

In [ ]:
IMG_PATH = '/home/igor/projects/Seed/data/mutation/session_000/'
CALIBR_WHITE_PATH = '/home/igor/projects/Seed/data/mutation/calibr/session_000_024_snapshot_cube.tiff'
CALIBR_BLACK_CAP_PATH = '/home/igor/projects/Seed/data/mutation/calibr/session_000_025_snapshot_cube.tiff'
CALIBR_BLACK_BACKGROUND_PATH = '/home/igor/projects/Seed/data/mutation/calibr/session_000_026_snapshot_cube.tiff'
PURPLE_IMG = '/home/igor/projects/Seed/data/mutation/session_000/session_000_046_snapshot_cube.tiff'

In [ ]:
class MutationHyperImg(hyper.HyperImg):
    
    def _get_tiff(self) -> np.ndarray:
        img = tiff.imread(self.path)
        bl_img = tiff.imread(CALIBR_BLACK_BACKGROUND_PATH)
        new_img = np.where(bl_img > img, 0, img - bl_img)
        wh_img = tiff.imread(CALIBR_WHITE_PATH)
        return new_img /(wh_img - bl_img)

    
    def _get_target_varible(self) -> str:
        if not set(IMG_PATH.split('/')).issubset(set(self.path.split('.')[0].split('/'))):
            raise NameError('Error in path')
        
        number = int(self.path.split('/')[-1].split('.')[0].split('_')[2])
        
        if number < 25 and number > 0:
            return '-4'  
        elif number > 24 and number < 27:
            return '-1' 
        elif number > 26 and number < 41:
            return 'wt' 
        elif number > 40 and number < 45:
            return '-1'
        elif number == 45:
            return 'light'
        elif number == 46:
            return 'purple'
        elif number == 47:
            return 'blue'
        else:
            return 'control'

In [ ]:
img_lst: list[MutationHyperImg] = hyper.get_list_hyper_img(path=IMG_PATH, 
                                                       class_name=MutationHyperImg,
                                                       filter=lambda x: x != 'control', 
                                                       threshold_value=28.27608638, 
                                                       target_varible_name= 'Mutation')

In [ ]:
img_lst[0]

In [ ]:
len(img_lst)

In [ ]:
plt.figure(figsize=(7,5))
plt.grid()
sns.lineplot(data=hyper.get_df_graphics_medians_wavelenght(img_lst), 
             x='Wavelength', y='Median', hue='Mutation')

In [ ]:
plt.figure(figsize=(7,5))
plt.grid()
sns.lineplot(data=hyper.get_df_graphics_medians_wavelenght(img_lst, PURPLE_IMG), 
             x='Wavelength', y='Median', hue='Mutation')

In [ ]:
pipe = Pipeline([('scaler', StandardScaler()), ('pca', PCA(n_components=5))])

In [ ]:
df = hyper.get_df_medians(img_lst)
df.head()

In [ ]:
X = df.drop(['Mutation'], axis = 1)
y = df[['Mutation']]
X = pipe.fit_transform(X)
pipe['pca'].explained_variance_ratio_

In [ ]:
plt.figure(figsize=(7,5))
sns.scatterplot(data=pd.DataFrame(hyper.get_df_2_pca(img_lst), 
                                  columns=['1', '2', 'Mutation']), x='1', y='2', hue='Mutation')

In [ ]:
plt.figure(figsize=(7,5))
sns.scatterplot(data=pd.DataFrame(hyper.get_df_2_pca(img_lst, 'purple'), 
                                  columns=['1', '2', 'Mutation']), x='1', y='2', hue='Mutation')